In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "20" 
os.environ["MKL_NUM_THREADS"] = "22"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import datetime as dt
import torch.nn.functional as F
from src import STD_DATA_DIR
from sklearn.preprocessing import MinMaxScaler

/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# Prédiction des ventes par réseau de neurones récurrents

Objectif : 
- Construire un réseau de neurones récurrents (LSTM) pour prédire les prochaines ventes sur quelques catégories
- Comparer les approches avec & sans concurrence
- Identifier les catégories pertinentes pour ce type d'approche

## I Données
#### I.A Importation des données & premier traitements 

In [ ]:
data = pd.read_csv(STD_DATA_DIR /'dt_for_nn.csv', sep = ';')
data.set_index(['product','date'], inplace = True)
data = data[data.index.get_level_values(1) > '2017-01-01']

In [ ]:
backfills_cols = ['prix_moy','prix_min_marche_moy','marge_srp','min_marche','min_mp','pump','marge_pump_avg']
for col in backfills_cols:
    data[col] = data[col].groupby(level = 0).fillna(method = 'bfill')
    data[col] = data[col].groupby(level = 0).fillna(method = 'ffill')

In [3]:
#del(data)
data = pd.read_csv(STD_DATA_DIR / 'filtered_dt_for_nn', sep = ';')

/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#data.to_csv(STD_DATA_DIR / 'filtered_dt_for_nn', sep = ';')

In [49]:
set(data ['Niv. 3'])

{'SOIN DU SIÈGE',
 'Chauffage et brumisateur ext',
 'Accessoires electro',
 'Divers Composants',
 'Jeux de Bain - Hochets',
 'Plant de légume',
 'ACCESSOIRES ET DETERGENTS',
 '***HS***Bougie Photophore',
 'Ventilateurs/Refroidissement',
 'Autre outil électroportatif',
 'Interrupteur - prise - douille',
 'Nourriture poisson',
 'Talc',
 'Classeur',
 'Couchage chien',
 "TV LED 4K de plus de 55''",
 'Voitures Miniatures - CS',
 'Peinture - Dessin - Coloriage',
 'Biberons',
 'Cages et accessoires rongeur',
 'Courant Porteur - répéteurs',
 'Ampoule fluorescente',
 'Cave multi-température (P)',
 'Accessoire de porte',
 'Tournevis - Visseuse électric',
 'Poeles wok sauteuse',
 'Pot crayon-Sous main-Corbeille',
 "TV LED FHD moins de 24''",
 'Poussette multiple',
 'CAble Jack',
 'Rangement chambre Adulte',
 'Robots radiocommandés',
 'Matériel soudage',
 'Accessoires disques durs',
 'Fenêtre abattante',
 'Thermomètre',
 'Etabli - tréteau',
 'Receveur douche',
 'Phytosanitaire',
 'Accessoire salle

In [48]:
data = pd.read_csv(STD_DATA_DIR / 'filtered_dt_for_nn', sep = ';')
data.set_index(['product','date'], inplace = True)

/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### I.B Filtrage des données

In [4]:

index ='Vente lisse' #index usuel utilisé pour le filtrage

def filter_by_level(data,fam, niv='Niv. 3'):
    return data[data[niv] == fam]

def add_sum(data, fam, niv= 'Niv. 3'):
    df = filter_by_level(data, fam, niv)
    sum_df = df.groupby(level = 1)[index].sum()
    return df.join(sum_df, rsuffix ='_somme')

def extract_data(data, fam, niv= 'Niv. 3'):
    df = add_sum(data, fam, niv)
    df['part_de_marché']  = df[index]/df[index+'_somme']
    return df

fam = 'Aspirateur souffleur'
level = 'Niv. 3'
df = add_sum(data, fam, niv=level)


In [5]:
#Scaling
cols_to_scale =['prix_moy','prix_min_marche_moy','marge_srp', 'pump']
scaler = MinMaxScaler(feature_range=(0,10))
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])


In [6]:
del(data)

### I.C Adaptation des données à Pytorch

In [7]:
# filtre produit:
nb_ventes_mini = 100 #Nombre minimal de ventes                                                        
len_min =  10 # Nombre de semaines minimales de ventes 



In [ ]:
def gener_torch_dataset(data,split_date, col_drop =['Unnamed: 0','Vente lisse','min_marche', 'Vente réelle','Niv. 1','Niv. 2','Niv. 3'], horizon = 3 ):
    target =100*(data[index]/data[index+'_somme']).shift(horizon).iloc[horizon:]
    y_train = torch.tensor(target[target.index < split_date].values, dtype = torch.float32)
    y_test = torch.tensor(target.values, dtype = torch.float32)
    
    
    X = data[horizon:]
    #X['truth'] = 100 * data[index]/data[index+'_somme']
    X.drop(columns = col_drop, inplace= True)
    X['pdM'] = 100 * data[index]/data[index+'_somme']
    
    n_item = X.shape[1]
    X_train =  torch.tensor(X[X.index < split_date].values, dtype= torch.float32).reshape(-1,1,n_item)
    X_test = torch.tensor(X.values, dtype= torch.float32).reshape(-1,1,n_item)
    return X_train, y_train, X_test,y_test, torch.tensor(X['pdM'], dtype = torch.float32)



In [ ]:
train_list = []
test_list = []
keeped_products = []
date_train = '2019-01-01'
products = set(df.index.get_level_values(0))
for prod in products:
    vt_prod = df.loc[prod].fillna(0)
    if sum(vt_prod[index]) > nb_ventes_mini:
        min_date = min(vt_prod[vt_prod[index] > 0].index)
        max_date = min(date_train, max(vt_prod[vt_prod[index] > 0].index))
        
        min_date = dt.datetime.strptime(min_date, '%Y-%m-%d')
        max_date = dt.datetime.strptime(max_date, '%Y-%m-%d')
        
        if (max_date - min_date).days // 7 > len_min:
            keeped_products.append(prod)
            X_train, y_train, X_test,y_test,ref = gener_torch_dataset(vt_prod[vt_prod.index > dt.datetime.strftime(min_date,'%Y-%m-%d' )], date_train)
            train_list.append((X_train,y_train))
            test_list.append((X_test,y_test,ref))
            
print('Nb de produits initial  %s ' % len(products))
print('Nb de ventes initiales %s ' % sum(df[index]))
print('Nb de produits conservés %s' %len(keeped_products))        
print('Nb de ventes conservées %s' % sum(df[index].loc[keeped_products]))

In [ ]:
X_train.shape

## II Modèle sans concurrence
#### II.A Réseau de neurones

In [8]:


class LSTM_nn(nn.Module):
    def __init__(self, n_param,n_hidden, n_layer):
        super().__init__()
        self.rnn = nn.LSTM(n_param,n_hidden,n_layer)
        self.hidden_layer = (torch.zeros(n_layer, 1,n_hidden),
                            torch.zeros(n_layer, 1 ,n_hidden))
        self.linear = nn.Linear(n_hidden,1)
        self.n_hidden = n_hidden
        
    def forward(self,x):
        output, self.hidden_layer = self.rnn(x, self.hidden_layer)
        return F.softplus(self.linear(output))
    
    def reinitialize(self):
        self.hidden_layer = (torch.normal(0,1,size=(n_layer, 1,self.n_hidden)),
                            torch.normal(0,1,size=(n_layer, 1,self.n_hidden)))


In [ ]:
#Constantes 

epochs =1000
learning_rate =  1e-3
n_hidden = 5
n_param = X_train.shape[2]
n_layer =2

model = LSTM_nn(n_param,n_hidden,n_layer)
loss_function = nn.L1Loss(reduction ='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#### II.B Entrainement

In [ ]:
from random import shuffle

err_train  = []
err_test =  []

for i in range(epochs):
    l=0
    s=0
    copy_train_list = train_list.copy()
    shuffle(copy_train_list)
    for X_train,y_train in copy_train_list:
        optimizer.zero_grad()
        model.reinitialize()
        y_pred = model(X_train)
        single_loss = loss_function(y_pred[-30:,0],y_train[-30:] )
        single_loss.backward()
        optimizer.step()
        l += single_loss.item()
        s += sum(y_train[-30:])
        
    l_test = 0
    s_t = 0
    l_ref = 0
    for X_test, y_test,ref  in test_list:
        model.reinitialize()
        y_pred = model(X_test)[-50:,0]
        single_loss = loss_function(y_pred,y_test[-50:] )
        l_test += single_loss.item()
        s_t+= sum(y_test[-50:])
        
        l_ref += loss_function(ref[-50:],y_test[-50:]).item()
    print('Training MAPE: %.4f  Testing MAPE: %.4f Ref : %.4F' %(l/s  , l_test/s_t,l_ref/s_t ))
    err_train.append(l)
    err_test.append(l_test)

In [ ]:
X_train,y_train = train_list[5]
model.reinitialize()
model(X_train)[:,0,0]

In [ ]:
X_train[:,0,:]

In [ ]:
len(copy_train_list)

In [ ]:
plt.figure(figsize=(12,7))
#plt.plot(lt_test_ref,'--', label =  'Test Error Reference', color = 'red')
#plt.plot(lt_train_ref,'--', label =  'Train Error Reference', color = 'blue')
plt.plot(err_test, label =  'Test Error Reference', color =  'red')
plt.plot(err_train, label =  'Train Error Reference', color =  'blue')
plt.legend()
plt.xlabel("Epoch")
#plt.yscale('log')
plt.show()

In [ ]:
X_test,y_test,_ = test_list[3]

In [ ]:
model.reinitialize()
y_pred = model(X_test)[-35:,0]
y_pred

In [ ]:
sum(torch.abs(y_test[-35:] - y_pred[-35,:]))/sum(y_test[-35:])

In [ ]:
sum(y_test[-35:])

In [ ]:
loss_function(y_test[-35:],y_pred[-35,:]).item()

In [ ]:
y_test

## III A taille d'historique fixée
#### III.A Construction des données

In [9]:
def gener_lagged_dataset(serie_prod, period, horizon, min_cons=10):
    N = len(serie_prod)
    l = []
    for i in range(N-period-horizon+1):
        X = torch.tensor(serie_prod.drop(columns = 'pdM').iloc[i:i+period, :].values, dtype = torch.float32).reshape(period,1,-1) 
        y = torch.tensor(serie_prod['pdM'].iloc[i+horizon:i+horizon+period], dtype = torch.float32).reshape(period,1,-1)
        if sum(y) >=min_cons:
            l.append([X,y])
    return l

def gener_dat(data, index, col_drop =['Unnamed: 0','Vente lisse','min_marche', 'Vente réelle','Niv. 1','Niv. 2','Niv. 3'], horizon=6):
    X = data.copy()
    X.drop(columns = col_drop, inplace= True)
    X['pdM'] = 100 * (data[index]/data[index+'_somme'])
    X['pdM_shift'] = X['pdM'].shift(horizon)
    return X.iloc[horizon:]
    

In [15]:
#paramètres 
len_min = 25
window =20
horizon = 4
date_train = '2019-01-01'
nb_ventes_mini = 100
n_calc = 10

In [11]:

l_train = []
l_valid =[]
keeped_products =[]

products = set(df.index.get_level_values(0))
for prod in products:
    vt_prod = df.loc[prod].fillna(0)
    if sum(vt_prod[index]) > nb_ventes_mini:
        min_date = min(vt_prod[vt_prod[index] > 0].index)
        max_date = min(date_train, max(vt_prod[vt_prod[index] > 0].index))
        
        min_date = dt.datetime.strptime(min_date, '%Y-%m-%d')
        max_date = dt.datetime.strptime(max_date, '%Y-%m-%d')

        if (max_date - min_date).days // 7 > len_min:
            keeped_products.append(prod)
            l_train += gener_lagged_dataset(gener_dat(vt_prod[vt_prod.index < dt.datetime.strftime(max_date,'%Y-%m-%d' )], index, horizon=horizon),window,horizon)
            l_valid += gener_lagged_dataset(gener_dat(vt_prod[vt_prod.index > (dt.datetime.strftime(max_date - dt.timedelta(days = 7*horizon),'%Y-%m-%d' ) )], index,horizon=horizon),window,horizon)
            
print('Nb de produits initial  %s ' % len(products))
print('Nb de ventes initiales %s ' % sum(df[index]))
print('Nb de produits conservés %s' %len(keeped_products))        
print('Nb de ventes conservées %s' % sum(df[index].loc[keeped_products]))

Nb de produits initial  104 
Nb de ventes initiales 21892.603540221062 
Nb de produits conservés 32
Nb de ventes conservées 17383.077999857356


In [12]:
#Constantes

epochs =200
learning_rate =  5e-3
n_hidden = 20
n_param = l_train[0][0].shape[2]
n_layer =1

model = LSTM_nn(n_param,n_hidden,n_layer)
loss_function = nn.L1Loss(reduction ='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from random import shuffle


err_train  = []
err_test =  []

for i in range(epochs):
    l=0
    s=0
    copy_train_list = l_train.copy()
    shuffle(copy_train_list)
    
    optimizer.zero_grad()
    
    loss = 0
    for X_train,y_train in copy_train_list:
        model.reinitialize()
        y_pred = model(X_train)
        single_loss = loss_function(y_pred[-n_calc:,0],y_train[-n_calc:,0] )
        
        loss +=single_loss
        l += single_loss.item()
        s += sum(y_train[-n_calc:,0])
    loss.backward()
    optimizer.step()
        
    l_test = 0
    s_t = 0
    l_ref = 0
    for X_test, y_test  in l_valid:
        model.reinitialize()
        y_pred = model(X_test)
        single_loss = loss_function(y_pred[-n_calc:,0],y_test[-n_calc:,0] )
        l_test += single_loss.item()
        s_t+= sum(y_test[-n_calc:,0])
    if i % 5 ==0:
        print('Training MAPE: %.4f  Testing MAPE: %.4f' %(100 * l/s  , 100 * l_test/s_t))
    err_train.append(l)
    err_test.append(l_test)

In [13]:
plt.figure(figsize=(12,7))
#plt.plot(lt_test_ref,'--', label =  'Test Error Reference', color = 'red')
#plt.plot(lt_train_ref,'--', label =  'Train Error Reference', color = 'blue')
plt.plot(list(map(lambda x: 100 * x/float(s_t), err_test)), label =  'Test Error Reference', color =  'red')
plt.plot(list(map(lambda x: 100 * x/float(s), err_train)), label =  'Train Error Reference', color =  'blue')
plt.legend()
plt.xlabel("Epoch")
plt.ylabel('MAPE')
#plt.yscale('log')
plt.show()

NameError: name 'err_test' is not defined

<Figure size 864x504 with 0 Axes>

### La Même, avec competition

In [37]:
# filtre produit:
nb_ventes_mini = 0 #Nombre minimal de ventes                                                        
len_min =  0 # Nombre de semaines minimales de ventes 


In [38]:

def gener_lagged_dataset_mat(serie_prod, period, horizon,matrix, index, min_cons=0):
    N = len(serie_prod)
    for i in range(N-period-horizon+1):
        X = torch.tensor(serie_prod.iloc[i:i+period, :].values, dtype = torch.float32).reshape(period,1,-1) 
        y = torch.tensor(serie_prod['pdM'].iloc[i+horizon:i+horizon+period], dtype = torch.float32).reshape(period,1,-1)
        if sum(y) >=min_cons:
            matrix[index+i].append([X,y])

def gener_dat(data, index, col_drop =['Unnamed: 0','Vente lisse','min_marche', 'Vente réelle','Niv. 1','Niv. 2','Niv. 3'], horizon=6):
    X = data.copy()
    X.drop(columns = col_drop, inplace= True)
    X['pdM'] = 100 * (data['Vente lisse']/data['Vente lisse'+'_somme'])
    X['pdM_shift'] = X['pdM'].shift(horizon)
    return X.iloc[horizon:]
    

In [39]:
date_start = dt.datetime.strptime( min(df.index.get_level_values(1)), '%Y-%m-%d')
date_end_training = dt.datetime.strptime(date_train, '%Y-%m-%d')
nb_weeks = (date_end_training - date_start).days //7


In [42]:
mat_train = [[] for i in range(nb_weeks)]
mat_test = [[] for i in range(54 + horizon)]

l_train = []
l_valid =[]
keeped_products =[]

products = set(df.index.get_level_values(0))
for prod in products:
    vt_prod = df.loc[prod].fillna(0)
    if sum(vt_prod[index]) > nb_ventes_mini:
        min_date = min(vt_prod[vt_prod[index] > 0].index)
        max_date = min(date_train, max(vt_prod[vt_prod[index] > 0].index))
    
        min_date = dt.datetime.strptime(min_date, '%Y-%m-%d')
        max_date = dt.datetime.strptime(max_date, '%Y-%m-%d')
        index_start = (min_date-date_start).days//7
        
        if (max_date - min_date).days // 7 > len_min:
            keeped_products.append(prod)
            red_prod = vt_prod[vt_prod.index <= dt.datetime.strftime(max_date,'%Y-%m-%d' )]
            red_prod = red_prod[red_prod.index >= dt.datetime.strftime(min_date,'%Y-%m-%d' )]
            
            valid_red_prod = vt_prod[vt_prod.index > (dt.datetime.strftime(date_end_training - dt.timedelta(days = 7*horizon),'%Y-%m-%d' ) )]

            gener_lagged_dataset_mat(gener_dat(red_prod, index, horizon=horizon),window,horizon, mat_train,index_start)
            gener_lagged_dataset_mat(gener_dat(valid_red_prod, index, horizon=horizon),window,horizon, mat_test,0)            
print('Nb de produits initial  %s ' % len(products))
print('Nb de ventes initiales %s ' % sum(df[index]))
print('Nb de produits conservés %s' %len(keeped_products))        
print('Nb de ventes conservées %s' % sum(df[index].loc[keeped_products]))

Nb de produits initial  104 
Nb de ventes initiales 21892.603540221062 
Nb de produits conservés 83
Nb de ventes conservées 21005.874692294365


In [47]:
#Constantes

epochs =2000
learning_rate =  5e-3
n_hidden = 20
n_param = 10
n_layer =1

model = LSTM_nn(n_param,n_hidden,n_layer)
loss_function = nn.L1Loss(reduction ='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
from random import shuffle


err_train_2  = []
err_test_2 =  []

for i in range(epochs):
    l=0
    s=0
    copy_train_list = mat_train.copy()
    shuffle(copy_train_list)
    
    optimizer.zero_grad()
    loss = 0
    for mat_step in copy_train_list:
        if len(mat_step) !=0:
            
            partial_sum= torch.tensor(mat_step[0][1])
            for i in range(1,len(mat_step)):
                partial_sum += mat_step[i][1]
        
            
            model.reinitialize()
            cp_mat_step = mat_step.copy()
            sum_prediction = torch.ones(partial_sum.shape)
            for X_train,_ in cp_mat_step:
                model.reinitialize()
                sum_prediction += model(X_train)

            shuffle(cp_mat_step)
            for (X_train,y_train) in cp_mat_step:
                model.reinitialize()
                y_pred = partial_sum * model(X_train)/sum_prediction
                single_loss = loss_function(y_pred[-n_calc:,0],y_train[-n_calc:,0] )
                loss += single_loss
                l += single_loss.item()
                s += sum(y_train[-n_calc:,0])
    loss.backward()
    optimizer.step()
            
    
    l_test = 0
    s_test = 0 
    for mat_step in mat_test: 
        if len(mat_step) !=0:
            partial_sum= torch.tensor(mat_step[0][1])
            for i in range(1,len(mat_step)):
                partial_sum += mat_step[i][1]
            optimizer.zero_grad()
            model.reinitialize()
            cp_mat_step = mat_step.copy()

            sum_prediction = torch.zeros(y_train.shape)
            for X_test,_ in cp_mat_step:
                model.reinitialize()
                sum_prediction += model(X_test)

            for (X_train,y_train) in cp_mat_step:
                model.reinitialize()
                y_pred = partial_sum * model(X_train)/sum_prediction
                single_loss = loss_function(y_pred[-n_calc:,0],y_train[-n_calc:,0] )
                l_test += single_loss.item()
                s_test += sum(y_train[-n_calc:,0])
    
    print('Training MAPE: %.4f Testing MAPE: %.4f' %(100 * l/s , 100 * l_test/s_test))
    err_train_2.append(l)
    err_test_2.append(l_test)

/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/ipykernel_launcher.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Training MAPE: 85.4388 Testing MAPE: 115.2496
Training MAPE: 85.1531 Testing MAPE: 114.9988
Training MAPE: 85.0788 Testing MAPE: 115.4934
Training MAPE: 84.5405 Testing MAPE: 115.2309
Training MAPE: 84.9282 Testing MAPE: 115.2025
Training MAPE: 84.4866 Testing MAPE: 114.7804
Training MAPE: 84.2930 Testing MAPE: 115.1383
Training MAPE: 84.5122 Testing MAPE: 114.8590
Training MAPE: 84.2360 Testing MAPE: 114.3299
Training MAPE: 83.9423 Testing MAPE: 114.5561
Training MAPE: 84.2792 Testing MAPE: 114.5532
Training MAPE: 83.7342 Testing MAPE: 114.3624
Training MAPE: 83.6859 Testing MAPE: 114.2583
Training MAPE: 83.5847 Testing MAPE: 113.8489
Training MAPE: 83.6070 Testing MAPE: 114.4449
Training MAPE: 83.2321 Testing MAPE: 114.0974
Training MAPE: 83.0975 Testing MAPE: 113.5963
Training MAPE: 82.8151 Testing MAPE: 113.3061
Training MAPE: 82.4069 Testing MAPE: 113.1362
Training MAPE: 82.5682 Testing MAPE: 112.8396
Training MAPE: 82.3789 Testing MAPE: 112.4771
Training MAPE: 82.0738 Testing MAP

Training MAPE: 52.9047 Testing MAPE: 70.7445
Training MAPE: 52.7747 Testing MAPE: 71.2753
Training MAPE: 52.7209 Testing MAPE: 70.4955
Training MAPE: 52.6421 Testing MAPE: 70.0164
Training MAPE: 52.7887 Testing MAPE: 70.1192
Training MAPE: 52.6168 Testing MAPE: 70.4968
Training MAPE: 52.7979 Testing MAPE: 70.8190
Training MAPE: 52.5610 Testing MAPE: 70.4857
Training MAPE: 52.4838 Testing MAPE: 70.5943
Training MAPE: 52.6600 Testing MAPE: 69.6564
Training MAPE: 52.5761 Testing MAPE: 69.2988
Training MAPE: 52.6151 Testing MAPE: 68.8509
Training MAPE: 52.5874 Testing MAPE: 69.4321
Training MAPE: 52.4168 Testing MAPE: 69.7640
Training MAPE: 52.4534 Testing MAPE: 70.5767
Training MAPE: 52.4424 Testing MAPE: 70.3448
Training MAPE: 52.1932 Testing MAPE: 70.5479
Training MAPE: 52.2329 Testing MAPE: 70.4333


KeyboardInterrupt: 

In [46]:
mat_step =mat_train[13]
partial_sum= torch.tensor(mat_step[0][1])
for i in range(1,len(mat_step)):
    partial_sum += mat_step[i][1]
partial_sum

/home/CDBDX/remy.garnier/.virtualenvs/python_3.6_TS/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


tensor([[[96.6887]],

        [[97.0137]],

        [[92.6562]],

        [[86.1099]],

        [[86.0978]],

        [[67.2631]],

        [[62.7099]],

        [[72.8341]],

        [[65.6269]],

        [[64.1968]],

        [[62.6434]],

        [[55.7985]],

        [[58.1935]],

        [[64.6675]],

        [[66.2367]],

        [[85.6285]],

        [[84.7043]],

        [[83.5637]],

        [[75.3938]],

        [[86.7936]]])